<a href="https://colab.research.google.com/github/fogg-lab/transcriptomics-data-query/blob/main/notebooks/GEO_data_retrieval_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEO data prep with `transcriptomics-data-query`

**Known issues**:
- The function `tdq.geo.map_probes_to_genes` is not guaranteed to work on all microarray platform technologies. This is due to differences in how the probe set annotation table is organized between different platform technologies.
- Other query functions, such as `tdq.geo.get_geo_clinical_characteristics`, can fail if the metadata for the study on GEO is not organized according to how this package expects.

I would appreciate if you could report any issues using the package that you might encounter.

## Setup

### 1) Install the Python package

In [ ]:
# Clone repository
!git clone https://github.com/fogg-lab/transcriptomics-data-query.git

# Install package
!pip install ./transcriptomics-data-query

### 2) Install R packages
*This will take up to 10 minutes*

In [ ]:
# Extra preliminary steps to install packages faster in Colab/Ubuntu, using bspm
!sudo add-apt-repository -y ppa:marutter/rrutter4.0
!sudo add-apt-repository -y ppa:c2d4u.team/c2d4u4.0+
!sudo apt-get update && sudo apt-get install -y python3-{dbus,gi,apt}
!wget https://github.com/Enchufa2/bspm/archive/refs/tags/v0.5.4.tar.gz
!sudo R CMD INSTALL v0.5.4.tar.gz
!echo "bspm::enable()" | sudo tee -a /etc/R/Rprofile.site

In [ ]:
# Install packages using the script, install_r_packages.R
!sudo Rscript ./transcriptomics-data-query/install_r_packages.R

# Install preprocessCore in single threaded mode to avoid threading bug on Colab
!git clone https://github.com/bmbolstad/preprocessCore.git
!R CMD INSTALL --configure-args="--disable-threading" ./preprocessCore

## GEO retrieval and preprocessing example

Prepare expression data from a GEO study. This process is similar for microarray and RNASeq data, with some minor differences. Refer to the [documentation](https://github.com/fogg-lab/transcriptomics-data-query/blob/main/DOCUMENTATION.md) for more information.

In [ ]:
### 1. Import packages

import GEOparse
import pandas as pd
import transcriptomics_data_query as tdq

In [ ]:
### 2. Obtain a GSE object for a GEO series accession using the GEOparse package

# Link: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE161750
accession = "GSE161750"
gse = GEOparse.get_GEO(accession)

In [ ]:
### 3. Download the raw data

# If the output_dir argument is not specified, it will be created in the working directory.
# For microarray data, this results in a directory of CEL.gz files.
# If the accession was an RNASeq study, a counts file would be downloaded instead.
tdq.geo.download_geo_expression_data(gse)

In [ ]:
### 4. Download and parse clinical characteristics

# Save in current working directory, ./GSE161750_clinical.tsv
clinical_file = f"{accession}_clinical.tsv"
tdq.geo.get_geo_clinical_characteristics(gse, output_file=clinical_file)

# View parsed clinical characteristics.
# Probably need to clean it up manually in a spreadsheet program.
pd.read_csv(clinical_file, sep='\t')

In [ ]:
### 5. Normalization (RMA for microarray, TMM for RNASeq)

# If the accession is microarray data, input_path should be a directory of CEL.gz files
norm_input_path = accession
norm_save_path = f"{accession}_expression_matrix.tsv"
tdq.preprocess.normalize(input_path=norm_input_path, output_file=norm_save_path)

# For RNASeq, input_path should be the raw counts file, and clinical_file must also be specified.
# norm_input_path = f"{accession}_expression_matrix.tsv"
# norm_save_path = f"{accession}_expression_matrix.tsv"
# tdq.preprocess.normalize(norm_input_path, norm_save_path, clinical_file)

# Read normalized expression matrix from file
expr_df = pd.read_csv(norm_save_path, sep="\t", index_col=0)

# Clean up sample names (extract "GSMxxxx")
expr_df = tdq.geo.clean_geo_sample_columns(expr_df)

# Save expression matrix with cleaned sample names
cleaned_save_path = norm_save_path  # overwrite old file
expr_df.to_csv(cleaned_save_path, sep="\t")

# Preview current expression matrix
print(f"Expression matrix for {accession} after normalization:")
expr_df.iloc[:8, :3]

In [ ]:
### 5b Map probes to genes (for microarray data only)
expr_df = tdq.geo.map_probes_to_genes(expr_df, gse)

# Overwrite previous expression matrix file
expr_df.to_csv(cleaned_save_path, sep="\t")

# Preview current expression matrix
print(f"Expression matrix for {accession} after mapping probes to genes:")
expr_df.iloc[:8, :3]

In [ ]:
### 6. Further processing

# The annotation table for GSE161750 (platform GPL23159) contains Ensembl gene and transcript IDs.
# Suppose we want to filter the expression matrix by genes in the olfactory transduction pathway,
# and want the genes listed as symbols. We can use the `preprocess` module to achieve this.

In [ ]:
### 6.1. Get gene symbols in the KEGG pathway for olfactory transduction
gene_set = "KEGG_OLFACTORY_TRANSDUCTION"
gene_set_symbols = tdq.preprocess.get_genes_from_msig_set(gene_set)
# Alternatively you can read genes from a text file (1 gene per line):
# gene_set_symbols = tdq.preprocess.get_genes_from_file("genes_of_interest.txt")

# Preview gene symbols in the list
gene_set_symbols[:10]

In [ ]:
# 6.2. Obtain the ensembl IDs for this set of genes
# The convert_genes function uses mygene.info to convert genes from one format to another.
# The function accepts 3 formats: symbol, Enseml ID (ensembl.gene), and Entrez ID (entrezgene).

# Convert symbols to Ensembl IDs
gene_set_ensembl = tdq.preprocess.convert_genes(
    genes=gene_set_symbols, in_format="symbol", out_format="ensembl.gene", species="human")

# Discard not found genes
gene_set_ensembl.dropna(inplace=True)

# Preview Ensembl IDs in the gene set
gene_set_ensembl.head()

In [ ]:
# 6.3. Select rows in `expr_df` that contain these genes
filtered_expression = tdq.preprocess.select_rows(expr_df, gene_set_ensembl)

# 6.5. Preview the filtered expression matrix
print(f"\nFiltered expression matrix for {accession} (Ensembl IDs)")
print(filtered_expression.iloc[:8, :3])

In [ ]:
# 6.4. Optionally, convert the Ensemble IDs to gene symbols
genes = filtered_expression.index
filtered_expression.index = tdq.preprocess.convert_genes(genes, "ensembl.gene", "symbol")

# Preview the matrisome expression matrix
print(f"\nFiltered expression matrix for {accession} (gene symbols)")
print(filtered_expression.iloc[:8, :3])